In [1]:
#using PyPlot
include("lib/all.jl");

In [40]:
N=2^26
ax = zeros(N)
ay = zeros(N)
az = zeros(N);

particles = make_particles(N);
t = DTree(particles, 12);

In [41]:
@time fill!(t.tree.nodes, Node());

  0.313662 seconds (5 allocations: 304 bytes)


In [42]:
@time fill!(t.tree.exps, NodeExp());

  0.188799 seconds (5 allocations: 240 bytes)


In [43]:
@time group!(t);

  2.989372 seconds (6 allocations: 208 bytes)


In [44]:
@time inform!(t);

  1.832113 seconds (40.90 M allocations: 622.788 MB, 19.71% gc time)


In [45]:
fill!(ax,0.0)
fill!(ay,0.0)
fill!(az,0.0);

In [46]:
@time interact!(t, 0.26, ax, ay, az, 1.0e-3);

  2.624350 seconds (1.25 M allocations: 19.002 MB)


In [47]:
@time collect!(t)

  0.515825 seconds (4 allocations: 160 bytes)


In [48]:
@time accel!(t, ax,ay,az)

  0.455607 seconds (15 allocations: 672 bytes)


In [49]:
mean(abs(ax))/1e5

330.78683998926317

In [52]:
mx,myd,e50,e90,e95,e99,m,s = perf(particles, ax,ay,az, 100, 1.0e-3)
print(myd)

mean(abs(rax)) = 3.3337262296028808e7
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,7.0,15.0,21.0,21.0,11.0,13.0,3.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [53]:
semilogx(10.^mx,myd,"-k")

LoadError: UndefVarError: semilogx not defined

In [54]:
e50

0.7699370245018224

In [55]:
e90

1.5502624688333895

In [56]:
e99

5.4911230475802935

In [33]:
function doall(t, ax,ay,az, alpha, eps2)
    total_mass = t.trees[1].total_mass
    for it in t.trees
        fill!(it.exps, NodeExp());
        it.total_mass = total_mass
    end
    tic()
    group!(t)
    t_group = toq()
    tic()
    inform!(t)
    t_inf = toq()
    fill!(ax,0.0)
    fill!(ay,0.0)
    fill!(az,0.0)
    tic()
    interact!(t, alpha, ax, ay, az, eps2)
    t_int = toq()
    tic()
    collect!(t)
    t_coll = toq()
    tic()
    accel!(t,ax,ay,az)
    t_acc = toq()
    t_group, t_inf, t_int, t_coll, t_acc
end

doall (generic function with 1 method)

In [34]:
doall(t, ax, ay, az, 0.22, 1.0e-3)

(0.038793923,0.024093969,0.334581141,0.007797423,0.006346059)

In [50]:
s_g = Float64[]
s_inf = Float64[]
s_int = Float64[]
s_coll = Float64[]
s_acc = Float64[]
s_all = Float64[]
for i in 1:10
    tic()
    t_group, t_inf, t_int, t_coll, t_acc = doall(t, ax, ay, az, 0.26, 1.0e-3);
    t_all = toq()
    append!(s_g, t_group)
    append!(s_inf, t_inf)
    append!(s_int, t_int)
    append!(s_coll, t_coll)
    append!(s_acc, t_acc)
    append!(s_all, t_all)
    @show i
end

i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10


In [51]:
@show mean(s_g)
@show mean(s_inf)
@show mean(s_int)
@show mean(s_coll)
@show mean(s_acc)
@show mean(s_all);

mean(s_g) = 2.9766350332
mean(s_inf) = 1.6830599148
mean(s_int) = 2.6663258053999996
mean(s_coll) = 0.5391061176999999
mean(s_acc) = 0.4404352171
mean(s_all) = 8.663435967200002


In [20]:
8.63-2.89

5.74

In [29]:
t.trees[1].num_nodes_used / length(t.trees[1].nodes)

0.77694057820735

In [30]:
sizeof(t.tree.nodes)/2^30

2.6350000202655792

In [31]:
length(t.tree.particles)*sizeof(t.tree.particles[1])/2^30

2.0

In [32]:
sizeof(t.tree.exps)/2^30

1.395000010728836

In [38]:
sizeof(ax)*4*2/2^30

4.0

In [39]:
2.6+2.0+1.4+4

10.0

In [40]:
64/10*2^26/2^20

409.6